## Selecting the fingerprints

In this notebook I'm going to select the fingerprints that store the information in a more appropriate manner for a CNN model of 1 dimension. To do that, I'm going to program a script that uses different fingerprints to train a CNN and I will chose the ones giving a model with the best accuracy. The model is build to be fast to speed up the process.

## Imports

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import EarlyStopping
import warnings

### Load data

In [3]:
pd.options.mode.chained_assignment = None

In [15]:
drugs = pd.read_pickle(os.path.join('res', 'pickles', 'drugbank_fp.pkl')).drop('FeatInvariants', axis=1) #I discard the Feature Invariants because they give an error I have to look into yet
display(drugs.head())
drugs.iloc[0]['Morgan2FP'].shape
drugs.info()
drugs.dropna(inplace=True)

,Molecule,atc_code_3,CID,atc_code_#,ConnInvariants,Morgan2FP,MACCSKeys,AtomPairFP,TopTorFP,AvalonFP,PubchemFP,CactvsFP
5,<rdkit.Chem.rdchem.Mol object at 0x0000017A392...,B,101041682.0,1,"[2246728737, 2245384272, 2245273601, 224672873...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, ..."
6,<rdkit.Chem.rdchem.Mol object at 0x0000017A392...,L,657181.0,7,"[2246728737, 2245384272, 847961216, 2246699815...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, ..."
13,<rdkit.Chem.rdchem.Mol object at 0x0000017A392...,L,5311128.0,7,"[2246728737, 2245273601, 2246728737, 224538427...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, ..."
25,<rdkit.Chem.rdchem.Mol object at 0x0000017A392...,R,NaN,11,"[2246728737, 2245273601, 2246728737, 224538427...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None
33,<rdkit.Chem.rdchem.Mol object at 0x0000017A392...,H,5311065.0,5,"[847957139, 2246699815, 864942730, 2245384272,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ...","[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, ...","[1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, ..."


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2938 entries, 5 to 14995
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Molecule        2934 non-null   object 
 1   atc_code_3      2938 non-null   object 
 2   CID             2900 non-null   float64
 3   atc_code_#      2938 non-null   int32  
 4   ConnInvariants  2934 non-null   object 
 5   Morgan2FP       2934 non-null   object 
 6   MACCSKeys       2934 non-null   object 
 7   AtomPairFP      2934 non-null   object 
 8   TopTorFP        2934 non-null   object 
 9   AvalonFP        2934 non-null   object 
 10  PubchemFP       2900 non-null   object 
 11  CactvsFP        2900 non-null   object 
dtypes: float64(1), int32(1), object(10)
memory usage: 286.9+ KB


## Functions

In [16]:
def select_fp(column_list):
    """
    Function that takes a list of columns and makes a train/test split for each of them with the column containing the labels as y.
    Then returns a dictionary with the column name as a key and the tuple with X_train, X_test, y_train, y_test as value.
    Input: list with column names
    Output: dictionary
    """
    splits_dic = {}
    for column in column_list:
        X = drugs[column]
        y = drugs['atc_code_#'] #atc_code_# drug_class_code	
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        splits_dic[column] = X_train, X_test, y_train, y_test
    return splits_dic

In [17]:
fp_list = list(drugs.columns[5:])
splits_dic = select_fp(fp_list)
print(type(splits_dic['Morgan2FP'][0]))
print(splits_dic['Morgan2FP'][0].shape)
print(splits_dic['Morgan2FP'][0])

<class 'pandas.core.series.Series'>
(2027,)
842      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
9958     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
11715    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3405     [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...
1030     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                               ...                        
8018     [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1358     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1496     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4639     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1072     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Morgan2FP, Length: 2027, dtype: object


In [18]:
def build_model(inshape, nclasses):
    """
    Function that builds a CNN model with non optimal conditions.
    Input: tuple with inner shape of arrays, integer with number of classes
    Output: a CNN model
    """
    model = Sequential()
    model.add(Conv1D(100, 9, activation='relu', kernel_initializer='he_uniform', input_shape=inshape))
    model.add(MaxPool1D(2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.5))
    model.add(Dense(nclasses, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [19]:
def reshape_and_build(splits_dic):
    """
    Function that takes the dictionary generated by "select_fp" and reshapes X_train to fit a CNN model.
    Then builds the CNN model using the function 'build_model'. Return both the reshaped arrays and the models.
    Input: dictionary with column names as keys and a tuple of arrays as values
    Output: dictionary with column names as keys and a tuple with both reshaped arrays and CNN models 
    """
    arrays_models_dic = {}
    for column, tup in splits_dic.items():
        print(column)
        X_train = np.array(list(tup[0]))
        print('Shape X_train: ', X_train.shape, type(X_train))
        display(X_train[0])
        X_test = np.array(list(tup[1]))
        n_classes = len(np.unique(tup[2]))
        print('Number of classes: ', n_classes)
        X_train= X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test= X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
        print('Reshaped X_train: ', X_train.shape)
        in_shape = X_train.shape[1:]
        print('In_shape: ', in_shape)
        print('Shape y_train: ', tup[2].shape)
        
        model = build_model(in_shape, n_classes)
        arrays_models_dic[column] = X_train, X_test, tup[2], tup[3], model

    return arrays_models_dic

In [20]:
array_model_dic= reshape_and_build(splits_dic)

Morgan2FP
Shape X_train:  (2027, 2048) <class 'numpy.ndarray'>


array([0, 0, 0, ..., 0, 0, 0])

Number of classes:  14
Reshaped X_train:  (2027, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (2027,)
MACCSKeys
Shape X_train:  (2027, 167) <class 'numpy.ndarray'>


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0])

Number of classes:  14
Reshaped X_train:  (2027, 167, 1)
In_shape:  (167, 1)
Shape y_train:  (2027,)
AtomPairFP
Shape X_train:  (2027, 2048) <class 'numpy.ndarray'>


...

Number of classes:  14
Reshaped X_train:  (2027, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (2027,)
PubchemFP
Shape X_train:  (2027, 898) <class 'numpy.ndarray'>


array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,

Number of classes:  14
Reshaped X_train:  (2027, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (2027,)
CactvsFP
Shape X_train:  (2027, 898) <class 'numpy.ndarray'>


array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,

Number of classes:  14
Reshaped X_train:  (2027, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (2027,)


In [21]:
def fit_model(arrays_models_dic):
    """
    Function that takes the dictionary generated by 'reshape_and_build' and fits the CNN model contained in the values with the X_train and y_train arrays also in the values.
    Then evaluates the model against X_test e y_test and returns the accuracy of each model.
    Input: dictionary with column names as keys and a tuple with reshaped arrays and CNN models
    Output: a dictionary with column names as keys and the accuracy obtained by the CNN model for that column
    """
    accuracies_dic = {}
    es = EarlyStopping(monitor='val_loss', patience=1)
    for column, tup in arrays_models_dic.items():
        print(f"Analysing {column}")
        tup[4].fit(tup[0], tup[2], epochs=10, batch_size=128, verbose=1, validation_split = 0.2, callbacks = [es])
        loss, acc = tup[4].evaluate(tup[1], tup[3], verbose=1)
        accuracies_dic[column] = f'{acc:.3f}'
    return accuracies_dic

## Selection of the best fingerprints

Select the columns containing fingerprints

In [25]:
fp_list = list(drugs.columns[5:])
fp_list

['Morgan2FP',
 'MACCSKeys',
 'AtomPairFP',
 'TopTorFP',
 'AvalonFP',
 'PubchemFP',
 'CactvsFP']

Run the functions to obtain the dictionary with the accuracies

In [26]:
# Note for the evaluator: This step may take long,
# you may want to run it partially and see that it works and then skip it and see the result below
# or decrease substantially the number of epochs
splits_dic = select_fp(fp_list)
array_model_dic= reshape_and_build(splits_dic)
accuracies_dic = fit_model(array_model_dic)

Morgan2FP
Shape X_train:  (2027, 2048) <class 'numpy.ndarray'>


array([0, 0, 0, ..., 0, 0, 0])

Number of classes:  14
Reshaped X_train:  (2027, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (2027,)
MACCSKeys
Shape X_train:  (2027, 167) <class 'numpy.ndarray'>


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0])

Number of classes:  14
Reshaped X_train:  (2027, 167, 1)
In_shape:  (167, 1)
Shape y_train:  (2027,)
AtomPairFP
Shape X_train:  (2027, 2048) <class 'numpy.ndarray'>


...

Number of classes:  14
Reshaped X_train:  (2027, 2048, 1)
In_shape:  (2048, 1)
Shape y_train:  (2027,)
PubchemFP
Shape X_train:  (2027, 898) <class 'numpy.ndarray'>


array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,

Number of classes:  14
Reshaped X_train:  (2027, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (2027,)
CactvsFP
Shape X_train:  (2027, 898) <class 'numpy.ndarray'>


array([1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0,

Number of classes:  14
Reshaped X_train:  (2027, 898, 1)
In_shape:  (898, 1)
Shape y_train:  (2027,)
Analysing Morgan2FP
Epoch 1/10
13/13 [==============================] - 19s 1s/step - loss: 3.7903 - accuracy: 0.1431 - val_loss: 2.4566 - val_accuracy: 0.1897
Epoch 2/10
13/13 [==============================] - 18s 1s/step - loss: 2.3756 - accuracy: 0.1647 - val_loss: 2.3429 - val_accuracy: 0.2488
Epoch 3/10
13/13 [==============================] - 14s 1s/step - loss: 2.2166 - accuracy: 0.2381 - val_loss: 2.2044 - val_accuracy: 0.3424
Epoch 4/10
13/13 [==============================] - 12s 928ms/step - loss: 2.0736 - accuracy: 0.2961 - val_loss: 2.1340 - val_accuracy: 0.3621
Epoch 5/10
28/28 [==============================] - 2s 68ms/step - loss: 2.0688 - accuracy: 0.3774
Analysing MACCSKeys
Epoch 1/10
13/13 [==============================] - 4s 135ms/step - loss: 2.8570 - accuracy: 0.1351 - val_loss: 2.3804 - val_accuracy: 0.2118
Epoch 2/10
13/13 [==============================] - 1s 

In [27]:
accuracies_dic

{'Morgan2FP': '0.377',
 'MACCSKeys': '0.352',
 'AtomPairFP': '0.231',
 'TopTorFP': '0.374',
 'AvalonFP': '0.198',
 'PubchemFP': '0.211',
 'CactvsFP': '0.182'}

With 100 epochs and a patiente of 3, the best accuracy is achieved with the Morgan fingerprints. These will be used to tune and train our CNN model.